In [1]:
import os
import sys
import math
import logging
import yaml
import h5py
from pathlib import Path

import kerastuner as kt

import numpy as np
import scipy as sp
import tensorflow as tf

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

import began
from began.logging import setup_vae_run_logging

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
# initialize random seed in numpy
np.random.seed(123454321)
# initialize random seed in tensorflow
tf.random.set_seed(123454321)

plot_dir = Path("../reports/figures").absolute()

cfg = {}

with open("../configs/training/vae_train_default.yaml") as f:
    cfg.update(yaml.load(f, Loader=yaml.FullLoader))

with open("../configs/models/vae_default.yaml") as f:
    cfg.update(yaml.load(f, Loader=yaml.FullLoader))

logging.info("""Working with GPU: {:s}""".format(str(tf.test.is_gpu_available())))

logging.info("""
Network parameters:
    Size of latent dimension: {:d}
    Batch size: {:d}
    Epochs: {:d}
""".format(cfg['lat_dim'], cfg['batch_size'], cfg['epochs']))

INFO:root:Working with GPU: True
INFO:root:
Network parameters:
    Size of latent dimension: 256
    Batch size: 8
    Epochs: 400



In [3]:
# set up logging
summary_writer = setup_vae_run_logging(cfg['lat_dim'], cfg['batch_size'], cfg['epochs'])

In [4]:
# Batch and shuffle the data
with h5py.File("../data/preprocessed/prepared.h5", 'r') as f:
    dset = f["cut_maps"]
    train_images = dset[...].astype(np.float32)

dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(train_images.shape[0]).map(tf.image.per_image_standardization)
test_dataset = dataset.take(100)
train_dataset = dataset.skip(100)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [5]:
def build_model(hp):
    """Builds a convolutional model."""
    lat_dim = hp.Int('lat_dim', 32, 256)
    kernel_size = hp.Choice('kernel_size', values=[3, 5])
    return began.CVAE(lat_dim, kernel_size)

In [6]:
class MyTuner(kt.Tuner):

    def run_trial(self, trial, train_ds):
        hp = trial.hyperparameters

        # Hyperparameters can be added anywhere inside `run_trial`.
        # When the first trial is run, they will take on their default values.
        # Afterwards, they will be tuned by the `Oracle`.
        train_ds = train_ds.batch(hp.Int('batch_size', 8, 32, default=8))
        print(type(train_ds))
        model = self.hypermodel.build(trial.hyperparameters)
        lr = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log', default=1e-3)
        optimizer = tf.keras.optimizers.Adam(beta_1=0.5, learning_rate=lr)
        epoch_loss_metric = tf.keras.metrics.Mean()

        @tf.function
        def run_train_step(data):
            with tf.GradientTape() as tape:
                loss = began.vae.compute_loss(model, data)
                gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            epoch_loss_metric.update_state(loss)
            return loss

        # `self.on_epoch_end` reports results to the `Oracle` and saves the
        # current state of the Model. The other hooks called here only log values
        # for display but can also be overridden. For use cases where there is no
        # natural concept of epoch, you do not have to call any of these hooks. In
        # this case you should instead call `self.oracle.update_trial` and
        # `self.oracle.save_model` manually.
        for epoch in range(3):
            print('Epoch: {}'.format(epoch))

            self.on_epoch_begin(trial, model, epoch, logs={})
            for batch, data in enumerate(train_ds):
                self.on_batch_begin(trial, model, batch, logs={})
                batch_loss = float(run_train_step(data))
                self.on_batch_end(trial, model, batch, logs={'loss': batch_loss})

                if batch % 30 == 0:
                    loss = epoch_loss_metric.result().numpy()
                    print('Batch: {}, Average Loss: {}'.format(batch, loss))

            epoch_loss = epoch_loss_metric.result().numpy()
            self.on_epoch_end(trial, model, epoch, logs={'loss': epoch_loss})
            epoch_loss_metric.reset_states()

In [7]:
tuner = MyTuner(
      oracle=kt.oracles.BayesianOptimization(
          objective=kt.Objective('loss', 'min'),
          max_trials=10),
      hypermodel=build_model,
      directory='results',
      project_name='began_custom_training_hp')

INFO:tensorflow:Reloading Oracle from existing project results/began_custom_training_hp/oracle.json
INFO:tensorflow:Reloading Oracle from existing project results/began_custom_training_hp/oracle.json
INFO:tensorflow:Reloading Tuner from results/began_custom_training_hp/tuner0.json
INFO:tensorflow:Reloading Tuner from results/began_custom_training_hp/tuner0.json


In [13]:
tuner.search(train_ds=train_dataset)

INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit


In [14]:
best_hps = tuner.get_best_hyperparameters()[0]
print(best_hps.values)

{'lat_dim': 131, 'kernel_size': 5, 'batch_size': 10, 'learning_rate': 0.00019071203207567507}


In [15]:
best_model = tuner.get_best_models()[0]